# Example of using the gnomAD genetic ancestry principal components analysis loadings and random forest classifier

Please read our [blog post](https://gnomad.broadinstitute.org/news/2021-09-using-the-gnomad-ancestry-principal-components-analysis-loadings-and-random-forest-classifier-on-your-dataset/) about important caveats to consider when using gnomAD ancestry principal components analysis (PCA) loadings and random forest classifier models on your own dataset.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#The-following-packages-are-required-for-this-example" data-toc-modified-id="The-following-packages-are-required-for-this-example-2">The following packages are required for this example</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-4">Data Loading</a></span><ul class="toc-item"><li><span><a href="#Define-file-paths" data-toc-modified-id="Define-file-paths-4.1">Define file paths</a></span><ul class="toc-item"><li><span><a href="#v3-example-paths" data-toc-modified-id="v3-example-paths-4.1.1">v3 example paths</a></span></li><li><span><a href="#v2-example-paths" data-toc-modified-id="v2-example-paths-4.1.2">v2 example paths</a></span></li></ul></li><li><span><a href="#Define-the-number-of-PCs-used-for-v2-and-v3-genetic-ancestry-group-classification" data-toc-modified-id="Define-the-number-of-PCs-used-for-v2-and-v3-genetic-ancestry-group-classification-4.2">Define the number of PCs used for v2 and v3 genetic ancestry group classification</a></span></li><li><span><a href="#Define-the-RF-minimum-probability-used-for-v2-and-v3-genetic-ancestry-group-classification" data-toc-modified-id="Define-the-RF-minimum-probability-used-for-v2-and-v3-genetic-ancestry-group-classification-4.3">Define the RF minimum probability used for v2 and v3 genetic ancestry group classification</a></span></li><li><span><a href="#Load-ONNX-models" data-toc-modified-id="Load-ONNX-models-4.4">Load ONNX models</a></span></li><li><span><a href="#Load-gnomAD-v3.1-loadings-Hail-Table-and-the-VariantDataset-to-apply-projection-and-genetic-ancestry-group-assignment-to" data-toc-modified-id="Load-gnomAD-v3.1-loadings-Hail-Table-and-the-VariantDataset-to-apply-projection-and-genetic-ancestry-group-assignment-to-4.5">Load gnomAD v3.1 loadings Hail Table and the VariantDataset to apply projection and genetic ancestry group assignment to</a></span></li><li><span><a href="#Load-gnomAD-v2.1-precomputed-v2-PCA-scores" data-toc-modified-id="Load-gnomAD-v2.1-precomputed-v2-PCA-scores-4.6">Load gnomAD v2.1 precomputed v2 PCA scores</a></span></li></ul></li><li><span><a href="#Perform-PC-projection-using-the-v3.1-PCA-loadings" data-toc-modified-id="Perform-PC-projection-using-the-v3.1-PCA-loadings-5">Perform PC projection using the v3.1 PCA loadings</a></span><ul class="toc-item"><li><span><a href="#Create-dense-MatrixTable-of-only-the-variants-found-in-the-loadings-Table" data-toc-modified-id="Create-dense-MatrixTable-of-only-the-variants-found-in-the-loadings-Table-5.1">Create dense MatrixTable of only the variants found in the loadings Table</a></span></li><li><span><a href="#We-recommend-filtering-to-entries-meeting-GQ,-DP-and-het-AB-'adj'-thresholds" data-toc-modified-id="We-recommend-filtering-to-entries-meeting-GQ,-DP-and-het-AB-'adj'-thresholds-5.2">We recommend filtering to entries meeting GQ, DP and het AB 'adj' thresholds</a></span></li><li><span><a href="#Checkpoint-dense-MT-for-PC-projection" data-toc-modified-id="Checkpoint-dense-MT-for-PC-projection-5.3">Checkpoint dense MT for PC projection</a></span></li><li><span><a href="#Project-test-dataset-genotypes-onto-gnomAD-v3.1-loadings-and-checkpoint-the-scores" data-toc-modified-id="Project-test-dataset-genotypes-onto-gnomAD-v3.1-loadings-and-checkpoint-the-scores-5.4">Project test dataset genotypes onto gnomAD v3.1 loadings and checkpoint the scores</a></span></li></ul></li><li><span><a href="#Assign-genetic-ancestry-group-using-ONNX-RF-model" data-toc-modified-id="Assign-genetic-ancestry-group-using-ONNX-RF-model-6">Assign genetic ancestry group using ONNX RF model</a></span><ul class="toc-item"><li><span><a href="#v3.1-RF-model" data-toc-modified-id="v3.1-RF-model-6.1">v3.1 RF model</a></span></li><li><span><a href="#v2.1-RF-model" data-toc-modified-id="v2.1-RF-model-6.2">v2.1 RF model</a></span></li></ul></li></ul></div>

## Introduction

In this notebook, we show examples for how to use the gnomAD genetic ancestry PCA loadings and random forest (RF) classifiers.
- For the example using the [v3 release files](https://gnomad.broadinstitute.org/downloads#v3-ancestry-classification), we assign the genetic ancestry for a test dataset of gnomAD v4 using the v3.1 PCA loadings and the v3.1 ONNX formatted RF model.
  - We start with the test dataset in raw VariantDataset (VDS) format and reduce it to only variants with v3.1 loading info using the following steps:
      - Split multiallelic variants.
      - Filter to variants found in the v3.1 loadings HailTable (HT).
      - Densify the VDS to a MatrixTable (MT).
      - Filter the MT to high quality genotypes (GQ >= 20; DP >= 10; and AB >= 0.2 for het calls).
  - Then, project the genotypes onto v3.1 PCs using the v3.1 loadings HT.
  - Assign the genetic ancestry using the v3.1 ONNX formatted RF model.
  - **NOTE: the reference genome for the v3.1 loadings HT is GRCh38.**
- For the example using the [v2 release files](https://gnomad.broadinstitute.org/downloads#v2-ancestry-classification), we use our precomputed v2.1 PCA scores to assign the genetic ancestry of the v2 samples with the v2.1 ONNX formatted RF model.
   - For use on your own data, you can use the same process as shown for v3.1 to obtain the projected PCs.
   - **NOTE: the reference genome for the v2.1 loadings HT is GRCh37.**

## The following packages are required for this example

In [1]:
!pip install onnxruntime onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00


In [2]:
!pip install hail

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.6/405.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install gnomad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.3 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of python-jsonschema-objects to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.7/313.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [4]:
!pip install gcsfs

In [27]:
!apt-get install openjdk-11-jdk
!pip install pyspark

# Set up Java environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
Suggested packages:
  libice-doc libsm-doc libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk openjdk-11-jre
  x11-utils
0 upgraded, 14 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,517 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubun

In [5]:
!git clone https://github.com/broadinstitute/gnomad_qc.git

Cloning into 'gnomad_qc'...
remote: Enumerating objects: 25551, done.
remote: Counting objects: 100% (3809/3809), done.
remote: Compressing objects: 100% (1028/1028), done.
remote: Total 25551 (delta 2944), reused 3490 (delta 2754), pack-reused 21742 (from 1)
Receiving objects: 100% (25551/25551), 12.69 MiB | 18.88 MiB/s, done.
Resolving deltas: 100% (19809/19809), done.


In [6]:
%cd gnomad_qc

/content/gnomad_qc


## Imports

In [7]:
import onnx
import hail as hl
from gnomad.sample_qc.ancestry import (
    apply_onnx_classification_model,
    assign_population_pcs,
)
from gnomad.utils.filtering import filter_to_adj

from gnomad_qc.v2.resources.basics import get_gnomad_meta
from gnomad_qc.v4.resources.basics import get_checkpoint_path

## Data Loading

In [8]:
read_if_exists = True

### Define file paths

#### v3 example paths

In [9]:
# v3.1 PCA loadings.
gnomad_v3_loadings = (
    "gs://gcp-public-data--gnomad/release/3.1/pca/gnomad.v3.1.pca_loadings.ht"
)

# v3.1 ONNX RF model.
gnomad_v3_onnx_rf = (
    "gs://gcp-public-data--gnomad/release/3.1/pca/gnomad.v3.1.RF_fit.onnx"
)

# Test dataset to apply projection and genetic ancestry group assignment to.
# This will be the path to your dataset VDS.
vds_to_project = "gs://gnomad/v4.0/raw/exomes/testing/gnomad_v4.0_test.vds"

# v3.1 output paths.
test_mt_output_path = get_checkpoint_path("example_gnomad_v3.1_ancestry_rf", mt=True)
test_scores_output_path = get_checkpoint_path("example_gnomad_v3.1_ancestry_rf.scores")
gnomad_v3_assignment_path = get_checkpoint_path(
    "example_gnomad_v3.1_ancestry_rf.assignment"
)

#### v2 example paths

In [10]:
# v2.1 ONNX RF model.
gnomad_v2_onnx_rf = (
    "gs://gcp-public-data--gnomad/release/2.1/pca/gnomad.r2.1.RF_fit.onnx"
)

# v2.1 output path.
gnomad_v2_assignment_path = get_checkpoint_path(
    "example_gnomad_v2.1_ancestry_rf.assignment"
)

### Define the number of PCs used for v2 and v3 genetic ancestry group classification

In [11]:
v3_num_pcs = 16
v2_num_pcs = 6

### Define the RF minimum probability used for v2 and v3 genetic ancestry group classification

In [12]:
v3_min_prob = 0.75
v2_min_prob = 0.9

### Load ONNX models

In [13]:
!pip install --upgrade google-cloud-storage
from google.colab import auth
auth.authenticate_user()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.19.0 requires humanize>=4.6.0, but you have humanize 1.1.0 which is incompatible.


In [14]:
from google.cloud import storage
import onnx

client = storage.Client()

def download_file_from_gcs(gcs_path, destination):
    bucket_name, blob_name = gcs_path.split("/", 3)[2:]
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(destination)

# Download the ONNX files
download_file_from_gcs(gnomad_v2_onnx_rf, 'gnomad_v2_onnx_rf.onnx')
download_file_from_gcs(gnomad_v3_onnx_rf, 'gnomad_v3_onnx_rf.onnx')

In [15]:
# Load the ONNX models
with open('gnomad_v2_onnx_rf.onnx', 'rb') as f:
    v2_onx_fit = onnx.load(f)

with open('gnomad_v3_onnx_rf.onnx', 'rb') as f:
    v3_onx_fit = onnx.load(f)

### Load gnomAD v3.1 loadings Hail Table and the VariantDataset to apply projection and genetic ancestry group assignment to

In [28]:
vds = hl.vds.read_vds(vds_to_project)
v3_loading_ht = hl.read_table(gnomad_v3_loadings)

FatalError: UnsupportedFileSystemException: No FileSystem for scheme "gs"

Java stack trace:
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at is.hail.io.fs.HadoopFSURL.<init>(HadoopFS.scala:76)
	at is.hail.io.fs.HadoopFS.parseUrl(HadoopFS.scala:88)
	at is.hail.io.fs.HadoopFS.parseUrl(HadoopFS.scala:85)
	at is.hail.io.fs.FS.open(FS.scala:574)
	at is.hail.io.fs.FS.open$(FS.scala:574)
	at is.hail.io.fs.HadoopFS.open(HadoopFS.scala:85)
	at is.hail.expr.ir.RelationalSpec$.readMe

### Load gnomAD v2.1 precomputed v2 PCA scores

As mentioned above, the v2 example will use our precomputed v2 PCA scores.

In [34]:
# Install PySpark
!pip install pyspark

# Set environment variable for SPARK_HOME
import os
import shutil

# Create a Spark directory in /usr/local
SPARK_HOME = "/usr/local/spark"
os.makedirs(SPARK_HOME, exist_ok=True)

# Download Spark if not present
if not os.path.exists(f"{SPARK_HOME}/jars"):
    os.makedirs(f"{SPARK_HOME}/jars", exist_ok=True)

# Function to download a file if it doesn't already exist
def download_file(url, destination):
    if not os.path.exists(destination):
        !curl -sSL {url} -o {destination}

# Download necessary JAR files
download_file(
    "https://search.maven.org/remotecontent?filepath=org/apache/hadoop/hadoop-aws/3.2.0/hadoop-aws-3.2.0.jar",
    f"{SPARK_HOME}/jars/hadoop-aws-3.2.0.jar"
)

download_file(
    "https://search.maven.org/remotecontent?filepath=com/amazonaws/aws-java-sdk-bundle/1.11.375/aws-java-sdk-bundle-1.11.375.jar",
    f"{SPARK_HOME}/jars/aws-java-sdk-bundle-1.11.375.jar"
)

# Configure Spark to use AWS credentials
spark_conf_file = f"{SPARK_HOME}/conf/spark-defaults.conf"
os.makedirs(f"{SPARK_HOME}/conf", exist_ok=True)

with open(spark_conf_file, 'w') as f:
    f.write("""### START: DO NOT EDIT, MANAGED BY: install-s3-connector.sh
spark.hadoop.fs.s3a.aws.credentials.provider=com.amazonaws.auth.profile.ProfileCredentialsProvider,com.amazonaws.auth.profile.ProfileCredentialsProvider,org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider
### END: DO NOT EDIT, MANAGED BY: install-s3-connector.sh
""")

# Set Spark home in environment variables
os.environ['SPARK_HOME'] = SPARK_HOME
os.environ['PATH'] += f":{SPARK_HOME}/bin"

print("Spark setup complete!")


Spark setup complete!


In [35]:
v2_meta_ht = get_gnomad_meta("exomes", full_meta=True)
v2_pcs_ht = v2_meta_ht.select(
    scores=hl.array([v2_meta_ht[f"PC{pc+1}"] for pc in range(v2_num_pcs)])
).select_globals()
v2_pcs_ht = v2_pcs_ht.filter(hl.is_defined(v2_pcs_ht.scores[0]))

FatalError: UnsupportedFileSystemException: No FileSystem for scheme "gs"

Java stack trace:
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at is.hail.io.fs.HadoopFSURL.<init>(HadoopFS.scala:76)
	at is.hail.io.fs.HadoopFS.parseUrl(HadoopFS.scala:88)
	at is.hail.io.fs.HadoopFS.parseUrl(HadoopFS.scala:85)
	at is.hail.io.fs.FS.open(FS.scala:574)
	at is.hail.io.fs.FS.open$(FS.scala:574)
	at is.hail.io.fs.HadoopFS.open(HadoopFS.scala:85)
	at is.hail.expr.ir.RelationalSpec$.readMe

## Perform PC projection using the v3.1 PCA loadings

### Create dense MatrixTable of only the variants found in the loadings Table

In [21]:
# Reduce variant data to only annotations needed for split and densify.
# This includes annotations needed for our standard genotype filter ('adj').
vds = hl.vds.VariantDataset(
    vds.reference_data, vds.variant_data.select_entries("LA", "LGT", "GQ", "DP", "LAD")
)

# Split multiallelics.
vds = hl.vds.split_multi(vds, filter_changed_loci=True)

# Filter to variants in the loadings Table.
vds = hl.vds.filter_variants(vds, v3_loading_ht)

# Densify VDS.
mt = hl.vds.to_dense_mt(vds)

NameError: name 'vds' is not defined

### We recommend filtering to entries meeting GQ, DP and het AB 'adj' thresholds

In [ ]:
mt = filter_to_adj(mt)

### Checkpoint dense MT for PC projection

In [ ]:
mt = mt.checkpoint(
    test_mt_output_path, overwrite=not read_if_exists, _read_if_exists=read_if_exists
)

### Project test dataset genotypes onto gnomAD v3.1 loadings and checkpoint the scores

In [ ]:
# Project new genotypes onto loadings.
v3_pcs_ht = hl.experimental.pc_project(
    mt.GT,
    v3_loading_ht.loadings,
    v3_loading_ht.pca_af,
)

# Checkpoint PC projection results.
v3_pcs_ht = v3_pcs_ht.checkpoint(
    test_scores_output_path,
    overwrite=not read_if_exists,
    _read_if_exists=read_if_exists,
)

2023-07-18 16:09:26.455 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


## Assign genetic ancestry group using ONNX RF model

### v3.1 RF model

In [ ]:
ht, model = assign_population_pcs(
    v3_pcs_ht,
    pc_cols=v3_pcs_ht.scores[:v3_num_pcs],
    fit=v3_onx_fit,
    min_prob=v3_min_prob,
    apply_model_func=apply_onnx_classification_model,
)
ht = ht.checkpoint(
    gnomad_v3_assignment_path,
    overwrite=not read_if_exists,
    _read_if_exists=read_if_exists,
)

ht.aggregate(hl.agg.counter(ht.pop))

INFO (gnomad.sample_qc.ancestry 369): Found the following sample count after population assignment: nfe: 378, oth: 32, afr: 28, amr: 60, eas: 42, sas: 49, asj: 25, fin: 35


{'afr': 28,
 'amr': 60,
 'asj': 25,
 'eas': 42,
 'fin': 35,
 'nfe': 378,
 'oth': 32,
 'sas': 49}

### v2.1 RF model

In [36]:
ht, model = assign_population_pcs(
    v2_pcs_ht,
    pc_cols=v2_pcs_ht.scores,
    fit=v2_onx_fit,
    min_prob=v2_min_prob,
    apply_model_func=apply_onnx_classification_model,
)
ht = ht.checkpoint(
    gnomad_v2_assignment_path,
    overwrite=not read_if_exists,
    _read_if_exists=read_if_exists,
)

ht.aggregate(hl.agg.counter(ht.pop))

NameError: name 'v2_pcs_ht' is not defined